# A Scalable and Responsive API for Fashion MNIST Image Classification with Cloud ML Engine

In an earlier notebook you trained a convolutional neural network model for image classification and saved it to Google Cloud Storage (GCS). In this notebook, you will deploy the trained model as an Application Programming Interface (API or a web service) for online predictions.

In [0]:
import os
#@markdown Enter  your GCP Project ID:
PROJECT = "" #@param {type: "string"}
#@markdown Enter  your GCP Storage Bucket ID:
BUCKET = "" #@param {type: "string"}
#@markdown OPTIONAL: Replace with your GCP Storage Bucket Region:
REGION = "us-central1" #@param {type:"string"}

MODEL_TYPE='cnn'       # convolutional neural network

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['MODEL_TYPE'] = MODEL_TYPE
os.environ['TFVERSION'] = '1.10'  # Tensorflow version

def start_tensorboard(logdir, url_file):
  get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(logdir))
  get_ipython().system_raw('lt --port 6006 >> {} 2>&1 &'.format(url_file))
  get_ipython().system('cat {}'.format(url_file))

def stop_tensorboard(url_file):
  get_ipython().system_raw("ps -Af  | grep -E 'tensorboard|lt --port' | awk '{print $2}' | xargs -I % kill -9 %")
  get_ipython().system_raw("rm {}".format(url_file))
  
from google.colab import auth
auth.authenticate_user()  

In [0]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## Deploying and predicting with model

Deploy the model:

In [0]:
%%bash
MODEL_NAME="fashion"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/fashion/trained_${MODEL_TYPE}/export/exporter | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION

The previous step of deploying the model can take a few minutes. If it is successful, you should see an output similar to this one:

<pre>
Created ml engine model [projects/qwiklabs-gcp-27eb45524d98e9a5/models/fashion].
Creating version (this might take a few minutes)......
...................................................................................................................done.
</pre>

Next, download a local copy of the Fashion MNIST dataset to use with Cloud ML Engine for predictions.

In [0]:
import tensorflow as tf
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
LABELS = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

To predict with the model, save one of the test images as a JavaScript Object Notation (JSON) file. Also, take a look at it as a graphic and notice the expected class value in the title.

In [0]:
HEIGHT=28
WIDTH=28

IMGNO=12 #CHANGE THIS to get different images

#Convert raw image data to a test.json file and persist it to disk
import json, codecs
jsondata = {'image': test_images[IMGNO].reshape(HEIGHT, WIDTH).tolist()}
json.dump(jsondata, codecs.open('test.json', 'w', encoding='utf-8'))

#Take a look at a sample image and the correct label from the test dataset
import matplotlib.pyplot as plt
plt.imshow(test_images[IMGNO].reshape(HEIGHT, WIDTH))
title = plt.title('{} / Class #{}'.format(LABELS[test_labels[IMGNO]], test_labels[IMGNO]))

Here's how the same image looks when it saved in the test.json file for use with the prediction API.

In [0]:
!cat test.json

Send the file to the prediction service and check whether the model you trained returns the correct prediction.

In [0]:
!gcloud ml-engine predict \
   --model=fashion \
   --version=${MODEL_TYPE} \
   --json-instances=./test.json

Here is what my prediction service returned based on a model that I trained with Cloud ML Engine. Notice that the model predicts a probability of roughly 0.87 for the sneaker (class #7).

<pre>
CLASSES  PROBABILITIES
7        [2.627301398661075e-08, 1.843199037843135e-09, 6.106044111220399e-06, 5.581538289334276e-07, 9.015485602503759e-08, 0.09837665408849716, 3.8953788816797896e-07, 0.8761420249938965, 0.0034074552822858095, 0.02206665836274624]
</pre>

<pre>
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>